In [16]:
import pandas as pd 
df = pd.read_csv('expanded_output.csv')
df.drop(columns=['Number'])

,Sex,%Red Pixel,%Green pixel,%Blue pixel,Hb,Anaemic
0,M,43.264176,30.838924,25.899587,6.297293,Yes
1,F,43.144832,30.171404,26.692997,8.608315,Yes
2,F,46.506491,27.430905,26.051133,9.713010,Yes
3,F,44.963982,30.519205,24.499161,4.809385,Yes
4,M,45.069466,31.089378,23.853518,8.995228,Yes
...,...,...,...,...,...,...
495,F,45.704839,27.557135,26.712476,13.615373,No
496,F,44.906272,30.375286,24.713653,11.594262,No
497,F,43.430623,29.777833,26.782023,12.407689,No
498,F,43.304680,29.805033,26.875941,12.102483,No


In [17]:
df['Sex'].unique()

array(['M', 'F', 'M ', 'F '], dtype=object)

In [18]:
X = df.drop(columns=['Anaemic', 'Number'])
y = df['Anaemic']

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

In [22]:
trf1 = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), [0]), remainder='passthrough'
)

In [23]:
from sklearn.preprocessing import StandardScaler
trf2 = make_column_transformer(
    (StandardScaler(), slice(0, 7))
)

In [35]:
from sklearn.linear_model import LogisticRegression
trf3 = RandomForestClassifier()

In [28]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trf1, trf2, trf3)

In [29]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  slice(0, 7, None))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [30]:
y_pred = pipe.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print('Accuracy score', accuracy_score(y_test, y_pred))

Accuracy score 1.0


In [32]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[54,  0],
       [ 0, 46]], dtype=int64)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       1.00      1.00      1.00        46

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [34]:
import pickle 
pickle.dump(pipe, open('Anemia.pkl', 'wb'))